In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import utils
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

#Facebook Prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

In [3]:
#!pip install utils

In [6]:
#utils.median_filter

In [3]:
wd=pd.read_csv('https://storage.googleapis.com/weather-data-processed-for-le-wagon/cleaned_weather_hourly_darksky.csv')

df=pd.read_csv('https://storage.googleapis.com/energy_consumption_903/df_all_avg_v1.csv')

In [14]:
wd_filt=wd[['time','temperature','windSpeed','precipType_rain']]
wd_filt.rename(columns={'time':'DateTime'},inplace=True)

/tmp/ipykernel_31051/2613541129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_filt.rename(columns={'time':'DateTime'},inplace=True)


In [17]:
df[['DateTime']].merge(wd_filt,on='DateTime',how='inner')

,DateTime,temperature,windSpeed,precipType_rain
0,2012-01-01 00:00:00,9.75,1.79,1.0
1,2012-01-01 00:00:00,9.75,1.79,1.0
2,2012-01-01 00:00:00,9.75,1.79,1.0
3,2012-01-01 00:00:00,9.75,1.79,1.0
4,2012-01-01 00:00:00,9.75,1.79,1.0
...,...,...,...,...
588013,2014-02-28 00:00:00,6.37,4.08,1.0
588014,2014-02-28 00:00:00,6.37,4.08,1.0
588015,2014-02-28 00:00:00,6.37,4.08,1.0
588016,2014-02-28 00:00:00,6.37,4.08,1.0


In [21]:
df.set_index('DateTime',inplace=True)

In [22]:
df5=df.loc[:,['Acorn_Group','KWH/hh']]
data_gla=df5.groupby(by=df5.index).mean()
data_gla

,KWH/hh
DateTime,
2012-01-01 00:00:00,0.231551
2012-01-01 00:30:00,0.249021
2012-01-01 01:00:00,0.230137
2012-01-01 01:30:00,0.222626
2012-01-01 02:00:00,0.228136
...,...
2014-02-27 22:00:00,0.343924
2014-02-27 22:30:00,0.335663
2014-02-27 23:00:00,0.269226


In [23]:
data_gla=data_gla.reset_index()
data_gla

,DateTime,KWH/hh
0,2012-01-01 00:00:00,0.231551
1,2012-01-01 00:30:00,0.249021
2,2012-01-01 01:00:00,0.230137
3,2012-01-01 01:30:00,0.222626
4,2012-01-01 02:00:00,0.228136
...,...,...
37868,2014-02-27 22:00:00,0.343924
37869,2014-02-27 22:30:00,0.335663
37870,2014-02-27 23:00:00,0.269226
37871,2014-02-27 23:30:00,0.236664


In [24]:
data_gla.rename(columns={"DateTime": "ds", "KWH/hh": "y"},inplace=True)
data_gla

,ds,y
0,2012-01-01 00:00:00,0.231551
1,2012-01-01 00:30:00,0.249021
2,2012-01-01 01:00:00,0.230137
3,2012-01-01 01:30:00,0.222626
4,2012-01-01 02:00:00,0.228136
...,...,...
37868,2014-02-27 22:00:00,0.343924
37869,2014-02-27 22:30:00,0.335663
37870,2014-02-27 23:00:00,0.269226
37871,2014-02-27 23:30:00,0.236664


In [25]:
##  UK bank holidays, took out Northern Ireland and Scotland as data is for London
holidays=pd.read_csv(os.path.join(os.getcwd(),'../raw_data/uk_bank_holidays.csv'))
holidays.rename(columns={'Type':'holiday','Bank holidays':'ds'},inplace=True)
holidays

,holiday,ds
0,Boxing Day,26/12/2012
1,Christmas Day,25/12/2012
2,Summer bank holiday,27/08/2012
3,Queen's Diamond Jubilee,06/05/2012
4,Spring bank holiday,06/04/2012
5,Early May bank holiday,05/07/2012
6,Easter Monday,04/09/2012
7,Good Friday,04/06/2012
8,New Year's Day,01/02/2012
9,Boxing Day,26/12/2013


In [26]:
holidays.loc[:,'ds']=pd.to_datetime(holidays['ds'],format="%d/%m/%Y")

In [27]:
temp = wd_filt['temperature'].interpolate(method='linear')

In [30]:
m = Prophet(holidays=holidays,changepoint_prior_scale=0.01).fit(data_gla)
m.add_regressor('temp', prior_scale=0.5, mode='multiplicative')

21:51:59 - cmdstanpy - INFO - Chain [1] start processing
21:52:14 - cmdstanpy - INFO - Chain [1] done processing


Exception: Regressors must be added prior to model fitting.

In [ ]:
future = m.make_future_dataframe(periods=1488, freq='H')
fcst = m.predict(future)
fig = m.plot(fcst)
fig = m.plot_components(fcst)